# Extracción de datos desde bases de datos

## Conexión a bases de datos relacionales con ODBC y SQLAlchemy

#### Tipos de módulos de conexión
Para desarrollar un proyecto en Python en el que se quiera implementar el uso "Bases de Datos SQL", se tienen distintos módulos que pueden ser útiles, por ejemplo "PYODBC" que es uno de ellos.

**ODBC:**

Open DataBase Connectivity (ODBC) es un estándar de acceso a las bases de datos desarrollado por SQL Access Group (SAG) en 1992. El objetivo de ODBC es hacer posible el acceder a cualquier dato desde cualquier aplicación, sin importar qué sistema de gestión de bases de datos (DBMS) almacené los datos.

**PYODBC:**

Pyodbc es un módulo de Python de código abierto que simplifica el acceso a las bases de datos "ODBC" desde Python, implementando el uso de la DB API 2.0 de una forma conveniente para Python. Pyodbc también es considerado como un controlador SQL para Python.

### Conexión a bases de datos SQL Server utilizando cursor

#### Configuracion de SQL Server

So just open the access to your 127.0.0.1:1433 in the SQL server Configuration Manager.

Steps:

- Start -> All Programs -> SQL Server Configuration Manager
- SQL Server Network Configuration -> Protocols for MSSQLSERVER -> TCP/IP (Enable it)
- TCP/IP -> Properties -> IP Addresses. Find 127.0.0.1 and change the "Enabled" to "Yes". You can do it for all the IPs if you want.

In [1]:
! conda install pymssql -y

Channels:

warning  libmamba Added empty dependency for problem type SOLVER_RULE_UPDATE

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package pymssql-2.1.4-py27h0c8e037_0 requires python >=2.7,<2.8.0a0, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ pin-1 is installable and it requires
│  └─ python 3.12.* , which can be installed;
└─ pymssql is not installable because there are no viable options
   ├─ pymssql 2.1.4 would require
   │  └─ python >=2.7,<2.8.0a0 , which conflicts with any installable versions previously reported;
   ├─ pymssql [2.1.4|2.2.5] would require
   │  └─ python >=3.10,<3.11.0a0 , which conflicts with any installable versions previously reported;
   ├─ pymssql 2.1.4 would require
   │  └─ python >=3.5,<3.6.0a0 , which conflicts with any installable versions previously reported;
   ├─ pymssql [2.1.4|2.1.5] would require
   │  └─ python >=3.6,<3.7.0a0 , which conflicts with any


 - defaults
Platform: win-64
Solving environment: ...working... failed


In [1]:
import pymssql

conn = pymssql.connect(
    server='DSANDOVALFLAVIO', 
    database='cf_etl'
)

In [2]:
# Crear el cursor y ver las tablas de la base de datos
cursor = conn.cursor()
cursor.execute('SELECT * FROM information_schema.tables')
cursor.fetchall()

[('cf_etl', 'dbo', 'dataset', 'BASE TABLE')]

In [3]:
query = 'SELECT top 3 * FROM cf_etl.dbo.dataset'
cursor.execute(query)
data = cursor.fetchall()
data

[('12/24/2023',
  'CursosDeProgramacion',
  'Oeste',
  '26148',
  '133468',
  '21979',
  '606'),
 ('11/11/2023',
  'AprendeCSharpFacil',
  'Norte',
  '34586',
  '340290',
  '49146',
  '495'),
 ('5/24/2024',
  'CodigoFacilitoEnEspanol',
  'Norte',
  '20337',
  '343710',
  '32831',
  '1655')]

### Conexión a bases de datos SQL Server utilizando ODBC y SQLAlchemy

In [ ]:
! conda install pyodbc -y

In [14]:
! conda install sqlalchemy -y

Channels:
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\flavi\anaconda3\envs\tch_cf_etl_env

  added / updated specs:
    - sqlalchemy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    greenlet-3.0.1             |  py312hd77b12b_0         219 KB
    sqlalchemy-2.0.30          |  py312h827c3e9_0         3.8 MB
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

  greenlet           pkgs/main/win-64::greenlet-3.0.1-py312hd77b12b_0 
  sqlalchemy         pkgs/main/win-64::sqlalchemy-2.0.30-py312h827c3e9_0 



Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


Drivers Disponibles

In [4]:
import pyodbc

pyodbc.drivers()

['SQL Server',
 'PostgreSQL ODBC Driver(ANSI)',
 'PostgreSQL ODBC Driver(UNICODE)',
 'ODBC Driver 17 for SQL Server']

In [5]:
from sqlalchemy import create_engine

driver = 'ODBC Driver 17 for SQL Server'
server = 'DSANDOVALFLAVIO'
database = 'cf_etl'

engine = create_engine(f'mssql+pyodbc://{server}/{database}?driver={driver}')

# Testear la conexión
try:
    conn = engine.connect()
    print('Conexión exitosa')
except Exception as e:
    print(f'Error en la conexión: {e}')

Conexión exitosa


In [6]:
import pandas as pd

df_table = pd.read_sql_table('dataset', engine)
df_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Fecha                  1000 non-null   object
 1   Nombre de la CampaÃ±a  1000 non-null   object
 2   Region                 1000 non-null   object
 3   Clicks                 1000 non-null   object
 4   Impresiones            1000 non-null   object
 5   Views                  1000 non-null   object
 6   Costo                  1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB


### Ejecución de consultas SQL para extraer datos específicos

In [8]:
query = """
SELECT * 
FROM cf_etl.dbo.dataset 
WHERE sCampaign = 'CursosDeProgramacion'
"""

df = pd.read_sql(
    sql=query, 
    con=engine,
    
    )
df

,Fecha,sCampaign,Region,Clicks,Impresiones,Views,Costo
0,12/24/2023,CursosDeProgramacion,Oeste,26148,133468,21979,606
1,3/12/2024,CursosDeProgramacion,Oeste,32254,206296,6448,726
2,3/20/2024,CursosDeProgramacion,Norte,7761,178949,47475,862
3,10/22/2023,CursosDeProgramacion,Este,42967,356762,7496,1722
4,4/22/2024,CursosDeProgramacion,Norte,13362,157531,47034,517
...,...,...,...,...,...,...,...
92,4/1/2024,CursosDeProgramacion,Oeste,29805,381302,19833,1152
93,5/12/2024,CursosDeProgramacion,Sur,49996,79199,38665,1603
94,5/7/2024,CursosDeProgramacion,Este,22068,435276,34668,577
95,10/31/2023,CursosDeProgramacion,Este,31943,248545,21555,882


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Fecha        97 non-null     object
 1   sCampaign    97 non-null     object
 2   Region       97 non-null     object
 3   Clicks       97 non-null     object
 4   Impresiones  97 non-null     object
 5   Views        97 non-null     object
 6   Costo        97 non-null     object
dtypes: object(7)
memory usage: 5.4+ KB


# Extracción de datos desde bases de datos NoSQL (MongoDB)

In [28]:
! conda install pymongo -y

Channels:
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\flavi\anaconda3\envs\tch_cf_etl_env

  added / updated specs:
    - pymongo


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cryptography-43.0.0        |  py312h89fc84f_0         1.4 MB
    dnspython-2.4.2            |  py312haa95532_0         480 KB
    pymongo-4.6.3              |  py312hd77b12b_0         1.2 MB
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  cryptography       pkgs/main/win-64::cryptography-43.0.0-py312h89fc84f_0 
  dnspython          pkgs/main/win-64::dnspython-2.4.2-py312haa95532_0 
  pymongo            pkgs/main/win-64::pymongo-4.6.3-py312hd77b12b_0 



Preparing transaction: ...working...

In [12]:
import pymongo

client = pymongo.MongoClient('mongodb://localhost:27017/')
mydb = client['cf_etl']
mycollection = mydb['dataset']

In [13]:
#obtener todos los registros de la colección
for doc in mycollection.find():
    print(doc)

{'_id': ObjectId('66e855253d0b6a5338b09e51'), 'Fecha': datetime.datetime(2023, 12, 24, 6, 0), 'Nombre de la Campaña': 'CursosDeProgramacion', 'Region': 'Oeste', 'Clicks': 26148, 'Impresiones': 133468, 'Views': 21979, 'Costo': 606}
{'_id': ObjectId('66e855253d0b6a5338b09e52'), 'Fecha': datetime.datetime(2023, 11, 11, 6, 0), 'Nombre de la Campaña': 'AprendeCSharpFacil', 'Region': 'Norte', 'Clicks': 34586, 'Impresiones': 340290, 'Views': 49146, 'Costo': 495}
{'_id': ObjectId('66e855253d0b6a5338b09e53'), 'Fecha': datetime.datetime(2024, 5, 24, 6, 0), 'Nombre de la Campaña': 'CodigoFacilitoEnEspanol', 'Region': 'Norte', 'Clicks': 20337, 'Impresiones': 343710, 'Views': 32831, 'Costo': 1655}
{'_id': ObjectId('66e855253d0b6a5338b09e54'), 'Fecha': datetime.datetime(2023, 9, 5, 6, 0), 'Nombre de la Campaña': 'AprendePythonFacil', 'Region': 'Sur', 'Clicks': 45332, 'Impresiones': 279000, 'Views': 15840, 'Costo': 991}
{'_id': ObjectId('66e855253d0b6a5338b09e55'), 'Fecha': datetime.datetime(2024, 6,

In [14]:
def read_collection_df(collection):
    return pd.DataFrame(list(collection.find()))

read_collection_df(mycollection)

,_id,Fecha,Nombre de la Campaña,Region,Clicks,Impresiones,Views,Costo
0,66e855253d0b6a5338b09e51,2023-12-24 06:00:00,CursosDeProgramacion,Oeste,26148,133468,21979,606
1,66e855253d0b6a5338b09e52,2023-11-11 06:00:00,AprendeCSharpFacil,Norte,34586,340290,49146,495
2,66e855253d0b6a5338b09e53,2024-05-24 06:00:00,CodigoFacilitoEnEspanol,Norte,20337,343710,32831,1655
3,66e855253d0b6a5338b09e54,2023-09-05 06:00:00,AprendePythonFacil,Sur,45332,279000,15840,991
4,66e855253d0b6a5338b09e55,2024-06-19 06:00:00,CodigoFacilitoEnEspanol,Este,36817,462135,42041,1784
...,...,...,...,...,...,...,...,...
995,66e855253d0b6a5338b0a234,2024-07-06 06:00:00,CursoDeJavaAvanzado,Sur,29588,40026,18668,330
996,66e855253d0b6a5338b0a235,2023-12-13 06:00:00,AprendeCSharpFacil,Sur,24523,238325,5276,195
997,66e855253d0b6a5338b0a236,2024-01-26 06:00:00,AprendeCSharpFacil,Este,22159,437721,44179,442
998,66e855253d0b6a5338b0a237,2024-01-17 06:00:00,AprendeJavaScriptYA,Sur,47854,359512,20689,1674


La línea de código `pd.DataFrame(list(collection.find()))` está realizando varias operaciones para convertir los documentos de una colección de MongoDB en un DataFrame de Pandas. Aquí está la explicación detallada de cómo funciona:

1. **`collection.find()`**:
   - `collection` es un objeto que representa una colección en una base de datos MongoDB.
   - El método `find()` se utiliza para realizar una consulta en la colección y recuperar todos los documentos. Si no se pasan parámetros a `find()`, se recuperan todos los documentos de la colección.
   - El resultado de `find()` es un cursor, que es un iterador que permite recorrer los documentos recuperados uno por uno.

2. **`list(collection.find())`**:
   - La función `list()` toma el cursor devuelto por `find()` y lo convierte en una lista de documentos. Cada documento en la lista es un diccionario de Python que representa un documento de MongoDB.
   - Esta conversión es necesaria porque el cursor no puede ser directamente pasado a `pd.DataFrame()`. La lista resultante contiene todos los documentos de la colección en forma de diccionarios.

3. **`pd.DataFrame(list(collection.find()))`**:
   - `pd.DataFrame()` es una función de la biblioteca Pandas que se utiliza para crear un DataFrame.
   - Al pasar la lista de diccionarios a `pd.DataFrame()`, Pandas convierte cada diccionario en una fila del DataFrame. Las claves de los diccionarios se convierten en los nombres de las columnas del DataFrame.
   - El resultado es un DataFrame de Pandas que contiene todos los documentos de la colección de MongoDB, con las columnas correspondientes a las claves de los diccionarios.

En resumen, esta línea de código recupera todos los documentos de una colección de MongoDB, los convierte en una lista de diccionarios y luego crea un DataFrame de Pandas a partir de esa lista. Esto permite trabajar con los datos de MongoDB utilizando las poderosas herramientas de análisis y manipulación de datos que ofrece Pandas.

#### Ejecución de consultas para extraer datos específicos en MongoDB

In [15]:
def read_collection_df(collection, query={}):
    return pd.DataFrame(list(collection.find(query)))

query = {'Region': 'Norte'}
read_collection_df(mycollection, query)

,_id,Fecha,Nombre de la Campaña,Region,Clicks,Impresiones,Views,Costo
0,66e855253d0b6a5338b09e52,2023-11-11 06:00:00,AprendeCSharpFacil,Norte,34586,340290,49146,495
1,66e855253d0b6a5338b09e53,2024-05-24 06:00:00,CodigoFacilitoEnEspanol,Norte,20337,343710,32831,1655
2,66e855253d0b6a5338b09e56,2023-08-22 06:00:00,AprendeJavaScriptYA,Norte,39359,440556,5551,205
3,66e855253d0b6a5338b09e57,2024-04-10 06:00:00,AprendeJavaScriptYA,Norte,24900,338969,8259,134
4,66e855253d0b6a5338b09e5c,2024-02-15 06:00:00,CursoDeJavaAvanzado,Norte,21897,400482,13804,1121
...,...,...,...,...,...,...,...,...
235,66e855253d0b6a5338b0a228,2024-03-15 06:00:00,MasterEnPython,Norte,24234,354569,31836,1937
236,66e855253d0b6a5338b0a229,2023-11-26 06:00:00,CursoDeJavaAvanzado,Norte,5016,23477,47768,456
237,66e855253d0b6a5338b0a22c,2023-12-04 06:00:00,DesarrolloWebCompleto,Norte,15509,349479,20643,463
238,66e855253d0b6a5338b0a232,2023-11-15 06:00:00,DesarrolloWebCompleto,Norte,5641,49815,15662,1359
